In [1]:
from model import CovidMmodel
from datamodule import CovidDataModule
import model
import datamodule
import matplotlib as plt
import torch
from pytorch_lightning import Trainer

c:\Users\dungf\anaconda3\envs\xray_deploy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Khởi tạo mô hình để train

In [2]:
model = CovidMmodel(input_shape = (3, 224, 224),
                    num_classes = 3,
                    learning_rate = 1e-3,
                    dropout = 0.25,
                    label_smoothing = 0.0)

data_module = CovidDataModule(train_dir="data/train",
                            image_sise=(224, 224),
                            val_dir="data/val",
                            test_dir="data/test",
                            batch_size=8,
                            num_workers=8)

c:\Users\dungf\anaconda3\envs\xray_deploy\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\dungf\anaconda3\envs\xray_deploy\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\dungf/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|██████████| 230M/230M [01:27<00:00, 2.78MB/s] 


Chưa thấy GPU

In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


Train

In [3]:
trainer = Trainer(accelerator='cpu', devices=1, max_epochs=10)

history = trainer.fit(model, data_module)
trainer.validate(model,data_module)
trainer.test(model, data_module)
torch.save(trainer, 'resnet18.pt')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | ResNet           | 60.2 M
1 | dropout           | Dropout          | 0     
2 | classifier        | Linear           | 3.0 K 
3 | cross_entropy     | CrossEntropyLoss | 0     
4 | aucroc            | MulticlassAUROC  | 0     
-------------------------------------------------------
60.2 M    Trainable params
0         Non-trainable params
60.2 M    Total params
240.783   Total estimated model params size (MB)


c:\Users\dungf\anaconda3\envs\xray_deploy\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
c:\Users\dungf\anaconda3\envs\xray_deploy\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/3393 [00:00<?, ?it/s] 

c:\Users\dungf\anaconda3\envs\xray_deploy\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Validation DataLoader 0:   0%|          | 0/678 [00:00<?, ?it/s]

Đánh giá sau train

In [9]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


NameError: name 'history' is not defined